Autograd: Engine in pytorch to automatically compute gradients for backpropogation

Simple one layer NN

In [3]:
import torch

#input tensor
x = torch.ones(5)

#output
y = torch.zeros(3)

w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [4]:
loss

tensor(1.0600, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)